In [ ]:
from config import load_conf

config = load_conf("./config.yaml")

In [ ]:
from slack_util import SlackUtil

slack = SlackUtil(token=config.slack.token)
channel_to_summary = config.slack.channels["any-channel"]

conversation_history = slack.client.conversations_history(
    channel=channel_to_summary,
    limit=20
)
chat_history = []
for message in reversed(conversation_history["messages"]):
    chat = slack.to_chat_entry(message)
    print(chat)
    chat_history.append(chat)



In [ ]:
# This is for ollama
# from langchain_community.llms import Ollama
# llm = Ollama(model="llama3")
# llm = Ollama(model="phi3")
# llm = Ollama(model="wizardlm2")

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(api_key=config.open_ai.api_key,
                 model="gpt-3.5-turbo")
                #  model="gpt-4o")

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system",
     """
     I'd like you to analyze the following conversation and identify all my pending action items that not resolved and still require my attention.
     Please format these action items as a bulleted list with clear and concise descriptions of what needs to be done.
     """),
    ("user",
     """
     {conversation}
     """),
])

In [ ]:
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()

chain = prompt | llm | output_parser
output = chain.invoke({"conversation": chat_history})
print(output)